**Лабораторная работа Вариант 5 Павлова Татьяна**

In [ ]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/train.csv")

In [ ]:
data

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,RhoHV,RhoHV_5x5_10th,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
0,1,3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
1,1,16,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
2,1,25,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
3,1,35,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
4,1,45,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13765196,1180945,38,9.0,33.0,19.5,25.5,36.5,33.0,20.5,28.0,36.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765197,1180945,42,9.0,33.0,21.0,30.5,37.0,36.5,22.0,33.5,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765198,1180945,47,9.0,29.5,10.0,26.0,30.5,31.0,16.5,26.0,33.0,1.041667,NaN,1.025000,1.051667,1.75,NaN,0.750,3.0000,13.379990,NaN,NaN,13.379990,8.636004
13765199,1180945,52,9.0,19.0,NaN,15.5,26.5,19.0,NaN,16.5,26.5,NaN,NaN,NaN,1.051667,NaN,NaN,NaN,2.8125,NaN,NaN,NaN,NaN,8.636004


In [ ]:
data = data.dropna(axis = 0, subset = ['Ref'])
data = data.dropna(axis = 0, subset = ['RhoHV'])
data.reset_index(drop=True, inplace=True)

In [ ]:
train_id = data[~np.isnan(data.Ref)].Id.unique()
new_data = data[np.in1d(data.Id, train_id)]
del train_id
data = new_data
data.shape

(4069799, 24)

In [ ]:
data.fillna(0.0, inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
data

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,RhoHV,RhoHV_5x5_10th,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
0,2,1,2.0,9.0,5.0,7.5,10.5,15.0,10.5,16.5,23.5,0.998333,0.998333,0.998333,0.998333,0.3750,-0.1250,0.3125,0.8750,1.059998,-1.410004,-0.350006,1.059998,1.016000
1,2,6,2.0,26.5,22.5,25.5,31.5,26.5,26.5,28.5,32.0,1.001667,0.981667,0.998333,1.005000,0.0625,-0.1875,0.2500,0.6875,0.000000,0.000000,0.000000,1.409988,1.016000
2,2,11,2.0,21.5,15.5,20.5,25.0,26.5,23.5,25.0,27.0,1.001667,0.995000,0.998333,1.001667,0.3125,-0.0625,0.3125,0.6250,0.349991,0.000000,-0.350006,1.759994,1.016000
3,2,16,2.0,18.0,14.0,17.5,21.0,20.5,18.0,20.5,23.0,0.995000,0.995000,0.998333,1.001667,0.2500,0.1250,0.3750,0.6875,0.349991,-1.059998,0.000000,1.059998,1.016000
4,2,21,2.0,24.5,16.5,21.0,24.5,24.5,21.0,24.0,28.0,0.998333,0.995000,0.998333,0.998333,0.2500,0.0625,0.1875,0.5625,-0.350006,-1.059998,-0.350006,1.759994,1.016000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069794,1180928,57,5.0,15.0,10.0,12.0,15.5,24.0,16.0,21.0,25.5,1.001667,0.991667,1.001667,1.015000,0.6875,-0.4375,0.2500,0.6875,-1.410004,-2.120010,0.699997,2.119995,2.794001
4069795,1180935,41,14.0,14.0,0.0,15.0,23.5,19.0,0.0,16.5,28.5,1.051667,0.000000,0.935000,1.011667,-0.6875,0.0000,-1.0000,0.6875,0.349991,0.000000,-5.630005,4.889999,0.508000
4069796,1180935,46,14.0,17.5,0.0,18.0,25.0,17.5,0.0,18.0,25.0,1.051667,0.000000,0.928333,1.051667,-1.3125,0.0000,-2.8750,3.0000,-2.820007,0.000000,-2.820007,4.229996,0.508000
4069797,1180945,47,9.0,29.5,10.0,26.0,30.5,31.0,16.5,26.0,33.0,1.041667,0.000000,1.025000,1.051667,1.7500,0.0000,0.7500,3.0000,13.379990,0.000000,0.000000,13.379990,8.636004


In [ ]:
data = data.groupby(['Id']).aggregate({'Expected': 'median',
                                            'radardist_km': 'median',
                                            'Ref': 'median',
                                           'Ref_5x5_10th': 'median',
                                           'Ref_5x5_50th': 'median',
                                           'Ref_5x5_90th': 'median',
                                           'RefComposite': 'median',
                                           'RefComposite_5x5_10th': 'median',
                                           'RefComposite_5x5_50th': 'median',
                                           'RefComposite_5x5_90th': 'median',
                                           'RhoHV': 'median',
                                           'RhoHV_5x5_10th': 'median',
                                           'RhoHV_5x5_50th': 'median',
                                           'RhoHV_5x5_90th': 'median',
                                           'Zdr': 'median',
                                           'Zdr_5x5_10th': 'median',
                                           'Zdr_5x5_50th': 'median',
                                           'Zdr_5x5_90th': 'median',
                                           'Kdp': 'median',
                                           'Kdp_5x5_10th': 'median',
                                           'Kdp_5x5_50th': 'median',
                                           'Kdp_5x5_90th': 'median',}
                                           )

In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
data

,Expected,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,RhoHV,RhoHV_5x5_10th,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th
0,1.016000,2.0,17.00,14.25,18.25,22.25,22.50,20.50,22.50,25.25,0.998333,0.995000,0.998333,0.998333,0.37500,0.15625,0.37500,0.68750,-0.175003,-1.235001,-0.350006,1.059998
1,4.064002,9.0,20.00,8.50,16.50,20.00,20.00,10.00,17.50,20.00,0.988333,0.000000,0.988333,1.051667,-1.12500,0.00000,0.50000,3.50000,7.029999,0.000000,0.000000,7.029999
2,3.225002,8.0,20.50,0.00,17.50,27.00,21.50,21.00,24.00,28.50,0.945000,0.948333,0.998333,1.005000,1.12500,-0.75000,0.50000,2.18750,-1.050003,-2.470001,-1.050003,4.459992
3,0.010000,10.0,32.50,30.75,34.00,37.00,35.50,32.50,35.00,38.25,0.921667,0.848333,0.938333,0.990000,0.53125,-1.06250,0.46875,2.53125,-2.000008,-5.590004,0.529991,7.599991
4,2.540001,12.0,20.00,16.50,19.50,23.00,20.00,16.50,19.50,23.00,0.988333,0.968333,0.995000,1.008333,0.56250,-0.43750,0.18750,0.93750,-1.340012,-2.470001,-0.350006,2.119995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518527,0.508000,3.0,8.50,0.00,4.00,11.00,18.00,12.50,16.50,21.75,0.983333,0.976667,0.996667,1.001667,0.37500,-0.71875,-0.12500,0.75000,0.000000,-0.705002,-0.349998,2.805000
518528,15.494008,15.0,35.00,17.00,34.50,43.50,35.00,17.00,34.50,45.00,0.981667,0.968333,0.988333,1.028333,0.18750,-0.68750,0.25000,1.50000,-0.350006,-2.820007,-0.350006,3.529999
518529,2.794001,5.0,20.25,13.75,17.75,24.00,24.75,21.00,24.00,27.25,0.998333,0.991667,0.998333,1.001667,0.71875,0.03125,0.46875,1.06250,-0.525002,-1.235001,0.174995,1.234993
518530,0.508000,14.0,15.75,0.00,16.50,24.25,18.25,0.00,17.25,26.75,1.051667,0.000000,0.931667,1.031667,-1.00000,0.00000,-1.93750,1.84375,-1.235008,0.000000,-4.225006,4.559998


In [ ]:
ydata = data.iloc[:, 0]
ydata

0          1.016000
1          4.064002
2          3.225002
3          0.010000
4          2.540001
            ...    
518527     0.508000
518528    15.494008
518529     2.794001
518530     0.508000
518531     8.636004
Name: Expected, Length: 518532, dtype: float64

In [ ]:
data = data.drop(columns = 'Expected', axis = 1)

In [ ]:
# Среднее значение
mmean = data.mean(axis=0)
# Стандартное отклонение
mstd = data.std(axis=0)
data  -=  mmean
data /= mstd

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, ydata, test_size=0.33, random_state=42 )

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

In [ ]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

In [ ]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               5632      
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 48,897
Trainable params: 48,897
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=1024, validation_split = 0.2, verbose=1)

Epoch 1/100
272/272 [==============================] - 2s 6ms/step - loss: 37466.0195 - mae: 26.6187 - val_loss: 42380.5078 - val_mae: 25.8507
Epoch 2/100
272/272 [==============================] - 1s 5ms/step - loss: 37368.3438 - mae: 27.1967 - val_loss: 42426.0039 - val_mae: 34.2327
Epoch 3/100
272/272 [==============================] - 1s 5ms/step - loss: 37338.3125 - mae: 27.3140 - val_loss: 42276.3438 - val_mae: 27.4056
Epoch 4/100
272/272 [==============================] - 1s 5ms/step - loss: 37255.2852 - mae: 27.1436 - val_loss: 42238.1055 - val_mae: 26.5933
Epoch 5/100
272/272 [==============================] - 1s 5ms/step - loss: 37164.0977 - mae: 27.2524 - val_loss: 42256.2812 - val_mae: 22.7172
Epoch 6/100
272/272 [==============================] - 1s 5ms/step - loss: 37008.2461 - mae: 27.0333 - val_loss: 41979.4102 - val_mae: 24.5553
Epoch 7/100
272/272 [==============================] - 1s 5ms/step - loss: 36858.8984 - mae: 26.9184 - val_loss: 41888.9727 - val_mae: 23.6494

In [ ]:
y_test.reset_index(drop=True, inplace=True)

In [ ]:
pred = model.predict(X_test)

In [ ]:
pred = pred.flatten()
np.corrcoef(pred, y_test)

array([[1.        , 0.40447369],
       [0.40447369, 1.        ]])